## Install if necessary torchnet and torch

!pip install torch --user
!pip install torchnet --user

## Load library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchnet import meter

from torch.optim import Adam

from algorithms.vae import VAE

## VAE

Learn more [here](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)

Code source [here](https://github.com/shib0li/VAE-torch)

# ![](https://miro.medium.com/max/1400/1*Q5dogodt3wzKKktE0v3dMQ@2x.png)

## Load data and split into train and test set

In [2]:
import torchvision
import torchvision.datasets as datasets

In [3]:
mnist_trainset = datasets.MNIST(root='./dataset', train=True, download=True, transform=None)

In [4]:
len(mnist_trainset)

60000

In [5]:
mnist_trainset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7F4C144FAEB0>, 5)

In [6]:
from PIL import Image

In [7]:
mnist_trainset[0][0].show()

In [8]:
mnist_trainset[7][0].show()

In [9]:
mnist_testset = datasets.MNIST(root='./dataset', train=False, download=True, transform=None)

In [10]:
mnist_testset[0][0].show()

In [11]:
np.array(mnist_testset[0][0]).shape

(28, 28)

In [12]:
print(np.array(mnist_testset[0][0])[0:10, 0:10])

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  84 185 159 151]
 [  0   0   0   0   0   0 222 254 254 254]
 [  0   0   0   0   0   0  67 114  72 114]]


In [13]:
X_train = np.array([
    np.array(data)/255 for data, _ in mnist_trainset
])
data_dim = 28*28 # each image is size 28 * 28

In [14]:
X_test = np.array([
    np.array(data)/255 for data, _ in mnist_testset
])

In [15]:
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)

In [16]:
if torch.cuda.is_available():
    print('You use GPU !')
    device = torch.device('cuda')
else:
    print('You use CPU !')
    device = torch.device('cpu')

You use GPU !


## Model configuration

In [17]:
in_dim = 28*28 # each image is size 28 * 28
encoder_width = 32
decoder_width = 32
latent_dim = 16

In [18]:
batch_size=256
max_epoch=30
lr= 0.001
weight_decay = 0.0075

In [19]:
data = next(iter(X_train))
data.size()


torch.Size([28, 28])

## Train

In [20]:
# config model
model = VAE(X_train, X_test, in_dim, encoder_width, decoder_width, latent_dim, device)
# train VAE
hist_loss = model.train(batch_size, max_epoch, lr, weight_decay)

100%|██████████| 30/30 [00:55<00:00,  1.86s/it, val_loss=141, epoch=29]


In [21]:
np.savetxt('./dataset/loss.csv', hist_loss, delimiter=',')

## Generate new data

In [22]:
X_generated = model.test2(1)

In [24]:
X_generated[0].size()

torch.Size([28, 28])

In [30]:
image = Image.fromarray((X_generated[0]*255).cpu().detach().numpy())
image.show()

In [31]:
X_generated = model.test2(50) # generer 50 images

In [32]:
image = Image.fromarray((X_generated[10]*255).cpu().detach().numpy())
image.show()

In [33]:
image = Image.fromarray((X_generated[20]*255).cpu().detach().numpy())
image.show()

In [34]:
image = Image.fromarray((X_generated[30]*255).cpu().detach().numpy())
image.show()

In [35]:
image = Image.fromarray((X_generated[45]*255).cpu().detach().numpy())
image.show()